In [31]:
import tensorflow as tf
import tensorflow.keras as keras
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import signal
from tqdm import tqdm
from numpy.fft import fft, fftshift
import matplotlib.pyplot as plt
import seaborn as sns
import random
from tqdm import tqdm
import optuna
from scipy.integrate import cumtrapz
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import optimizers
import tensorflow_addons as tfa
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM,Bidirectional,Dropout
from keras.layers import LSTM
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from tensorflow.keras.utils import to_categorical
from keras.callbacks import EarlyStopping, ModelCheckpoint,ReduceLROnPlateau
from sklearn.model_selection import KFold,StratifiedKFold
from numpy.random import seed
from tensorflow.keras.layers import Activation, Conv2D, MaxPooling2D, Flatten, GlobalAveragePooling1D
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from statistics import mean, stdev
import warnings
warnings.filterwarnings(action='ignore')

print(tf.__version__) 

2.5.0


#### Data loading

In [32]:
train=pd.read_csv('./train_features.csv')
train_labels=pd.read_csv('./train_labels.csv')
test=pd.read_csv('./test_features.csv')

submission=pd.read_csv('./sample_submission.csv')

pd.options.display.max_columns=50

In [33]:
train.shape

(1875000, 8)

#### Feature engneering

#####  가속도, 자이로, (자이로-가속도) 센서값을 에너지로 표현

In [34]:
train['acc_Energy']=(train['acc_x']**2+train['acc_y']**2+train['acc_z']**2)**(1/3)
test['acc_Energy']=(test['acc_x']**2+test['acc_y']**2+test['acc_z']**2)**(1/3)

train['gy_Energy']=(train['gy_x']**2+train['gy_y']**2+train['gy_z']**2)**(1/3)
test['gy_Energy']=(test['gy_x']**2+test['gy_y']**2+test['gy_z']**2)**(1/3)

train['gy_acc_Energy']=((train['gy_x']-train['acc_x'])**2+(train['gy_y']-train['acc_y'])**2+(train['gy_z']-train['acc_z'])**2)**(1/3)
test['gy_acc_Energy']=((test['gy_x']-test['acc_x'])**2+(test['gy_y']-test['acc_y'])**2+(test['gy_z']-test['acc_z'])**2)**(1/3)

In [35]:
train.shape

(1875000, 11)

###### id별 데이터는 0.02초마다 측정된 값들이기 때문에 이전 시간 대비 변화량 적용

In [8]:
# dt=0.02 
# def jerk_signal(signal): 
#         return np.array([(signal[i+1]-signal[i])/dt for i in range(len(signal)-1)])

In [9]:
# train_dt=[]
# for i in tqdm(train['id'].unique()):
#     temp=train.loc[train['id']==i]
#     for v in train.columns[2:]:
#         values=jerk_signal(temp[v].values)
#         values=np.insert(values,0,0)
#         temp.loc[:,v+'_dt']=values
#     train_dt.append(temp)

100%|██████████| 3125/3125 [00:47<00:00, 65.73it/s]


In [10]:
# test_dt=[]
# for i in tqdm(test['id'].unique()):
#     temp=test.loc[test['id']==i]
#     for v in train.columns[2:]:
#         values=jerk_signal(temp[v].values)
#         values=np.insert(values,0,0)
#         temp.loc[:,v+'_dt']=values
#     test_dt.append(temp)

100%|██████████| 782/782 [00:10<00:00, 74.85it/s]


## 푸리에 변환

In [11]:
# from scipy import fftpack
# from numpy.fft import *

# def fourier_transform_one_signal(t_signal):
#     complex_f_signal= fftpack.fft(t_signal)
#     amplitude_f_signal=np.abs(complex_f_signal)
#     return amplitude_f_signal

In [12]:
# train=pd.concat(train_dt)
# train

,id,time,acc_x,acc_y,acc_z,gy_x,gy_y,gy_z,acc_Energy,gy_Energy,gy_acc_Energy,acc_total,acc_x_dt,acc_y_dt,acc_z_dt,gy_x_dt,gy_y_dt,gy_z_dt,acc_Energy_dt,gy_Energy_dt,gy_acc_Energy_dt,acc_total_dt
0,0,0,1.206087,-0.179371,-0.148447,-0.591608,-30.549010,-31.676112,1.146962,12.465436,12.427938,1.228355,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0,1,1.287696,-0.198974,-0.182444,0.303100,-39.139103,-24.927216,1.200703,12.913284,12.865692,1.315689,4.080495,-0.980114,-1.699854,44.735403,-429.504677,337.444793,2.687024,22.392358,21.887693,4.366731
2,0,2,1.304609,-0.195114,-0.253382,-3.617278,-44.122565,-25.019629,1.217403,13.725729,13.692643,1.343234,0.845632,0.192961,-3.546937,-196.018888,-249.173073,-4.620631,0.835012,40.622253,41.347563,1.377227
3,0,3,1.293095,-0.230366,-0.215210,2.712986,-53.597843,-27.454013,1.209981,15.374021,15.314907,1.330969,-0.575711,-1.762585,1.908626,316.513181,-473.763910,-121.719195,-0.371100,82.414636,81.113199,-0.613248
4,0,4,1.300887,-0.187757,-0.222523,4.286707,-57.906561,-27.961234,1.211254,16.074363,16.017964,1.333070,0.389598,2.130453,-0.365665,78.686055,-215.435892,-25.361098,0.063656,35.017060,35.152822,0.105059
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1874995,3124,595,-0.712530,-0.658357,0.293707,-29.367857,-104.013664,-76.290437,1.009050,25.963234,25.897316,1.013606,1.484646,0.303666,0.800069,-150.644663,-34.630282,-8.380088,-0.679712,8.387109,8.432977,-1.027613
1874996,3124,596,-0.683037,-0.658466,0.329223,-30.149089,-101.796809,-76.625087,1.002827,25.784692,25.722482,1.004243,1.474659,-0.005442,1.775771,-39.061611,110.842743,-16.732496,-0.311171,-8.927089,-8.741727,-0.468141
1874997,3124,597,-0.664730,-0.666625,0.364114,-27.873095,-98.776072,-79.365125,1.006239,25.628060,25.572145,1.009373,0.915321,-0.407957,1.744566,113.799702,151.036858,-137.001896,0.170620,-7.831611,-7.516832,0.256510
1874998,3124,598,-0.630534,-0.682565,0.373696,-23.636550,-99.139495,-80.259478,1.001038,25.626266,25.573288,1.001557,1.709833,-0.796984,0.479107,211.827245,-18.171144,-44.717652,-0.260074,-0.089713,0.057150,-0.390820


In [13]:
# fft=[]
# for i in tqdm(train['id'].unique()):
#     temp=train.loc[train['id']==i]
#     for i in train.columns[2:8]:
#         temp[i]=fourier_transform_one_signal(temp[i].values)
#     fft.append(temp)
# train=pd.concat(fft)

100%|██████████| 3125/3125 [00:11<00:00, 266.41it/s]


In [14]:
# test=pd.concat(test_dt)
# test

,id,time,acc_x,acc_y,acc_z,gy_x,gy_y,gy_z,acc_Energy,gy_Energy,gy_acc_Energy,acc_total,acc_x_dt,acc_y_dt,acc_z_dt,gy_x_dt,gy_y_dt,gy_z_dt,acc_Energy_dt,gy_Energy_dt,gy_acc_Energy_dt,acc_total_dt
0,3125,0,-0.628100,-0.160155,0.151487,49.665357,88.435961,13.597668,0.762377,21.878437,21.938882,0.665663,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,3125,1,-0.462548,0.012462,-0.053726,56.953059,96.185341,16.278458,0.600917,23.367908,23.399763,0.465824,8.277606,8.630860,-10.260605,364.385137,387.469010,134.039504,-8.072977,74.473561,73.044041,-9.991947
2,3125,2,-0.363481,-0.091789,-0.130004,29.557396,93.836453,13.329043,0.539978,21.440856,21.471510,0.396793,4.953330,-5.212527,-3.813919,-1369.783175,-117.444408,-147.470749,-3.046963,-96.352572,-96.412634,-3.451554
3,3125,3,-0.351750,-0.239870,-0.193053,23.686172,88.608721,13.449771,0.602338,20.482783,20.533967,0.467477,0.586560,-7.404041,-3.152460,-293.561181,-261.386607,6.036401,3.118006,-47.903682,-46.877152,3.534209
4,3125,4,-0.312934,-0.123762,-0.318621,20.410071,85.327707,13.884912,0.598853,19.908061,19.944125,0.463426,1.940804,5.805371,-6.278395,-163.805080,-164.050703,21.757050,-0.174231,-28.736096,-29.492133,-0.202539
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
469195,3906,595,0.104191,-0.784979,0.639513,-10.475346,14.095361,-190.358982,1.011867,33.185050,33.266220,1.017853,-3.361947,3.580749,0.006738,-771.499131,-849.527466,-8.671145,-2.122263,-9.277974,-9.636798,-3.235578
469196,3906,596,0.103297,-0.758954,0.615687,-25.360272,-8.523018,-180.393291,0.988451,32.158563,32.227841,0.982726,-0.044724,1.301266,-1.191327,-744.246300,-1130.918984,498.284561,-1.170808,-51.324317,-51.918937,-1.756341
469197,3906,597,0.128294,-0.749389,0.586184,-27.917723,-23.186245,-162.624160,0.973171,30.280197,30.339698,0.960028,1.249868,0.478247,-1.475149,-127.872573,-733.161359,888.456533,-0.763970,-93.918343,-94.407192,-1.134905
469198,3906,598,0.104130,-0.692731,0.573397,-27.847980,-30.407555,-138.761676,0.935801,27.569491,27.623600,0.905265,-1.208173,2.832870,-0.639335,3.487160,-361.065463,1193.124196,-1.868499,-135.535264,-135.804895,-2.738180


## 데이터셋 나누기

In [36]:
acc=train[["acc_x","acc_y","acc_z"]]
gy=train[["gy_x","gy_y","gy_z"]]

In [38]:
# acto=train["acc_total"]

## 스케일링 진행

In [39]:
train["acc_x_norm"]=(train["acc_x"]-np.nanmean(acc))/np.nanstd(acc)
train["acc_y_norm"]=(train["acc_y"]-np.nanmean(acc))/np.nanstd(acc)
train["acc_z_norm"]=(train["acc_z"]-np.nanmean(acc))/np.nanstd(acc)

train["gy_x_norm"]=(train["gy_x"]-np.nanmean(gy))/np.nanstd(gy)
train["gy_y_norm"]=(train["gy_y"]-np.nanmean(gy))/np.nanstd(gy)
train["gy_z_norm"]=(train["gy_z"]-np.nanmean(gy))/np.nanstd(gy)

#train["acc_total_norm"]=(train["acc_total"]-mean(acto))/stdev(acto)

In [40]:
test["acc_x_norm"]=(test["acc_x"]-np.nanmean(acc))/np.nanstd(acc)
test["acc_y_norm"]=(test["acc_y"]-np.nanmean(acc))/np.nanstd(acc)
test["acc_z_norm"]=(test["acc_z"]-np.nanmean(acc))/np.nanstd(acc)

test["gy_x_norm"]=(test["gy_x"]-np.nanmean(gy))/np.nanstd(gy)
test["gy_y_norm"]=(test["gy_y"]-np.nanmean(gy))/np.nanstd(gy)
test["gy_z_norm"]=(test["gy_z"]-np.nanmean(gy))/np.nanstd(gy)

#test["acc_total_norm"]=(test["acc_total"]-mean(acto))/stdev(acto)

In [41]:
train=train.drop(columns=["acc_x", "acc_y", "acc_z",
                   "gy_x", "gy_y", "gy_z"])

In [42]:
test=test.drop(columns=["acc_x", "acc_y", "acc_z",
                   "gy_x", "gy_y", "gy_z"])

In [43]:
train

,id,time,acc_Energy,gy_Energy,gy_acc_Energy,acc_x_norm,acc_y_norm,acc_z_norm,gy_x_norm,gy_y_norm,gy_z_norm
0,0,0,1.146962,12.465436,12.427938,2.189750,0.008884,0.057563,-0.004476,-0.441997,-0.458459
1,0,1,1.200703,12.913284,12.865692,2.318213,-0.021973,0.004047,0.008591,-0.567454,-0.359892
2,0,2,1.217403,13.725729,13.692643,2.344835,-0.015898,-0.107618,-0.048665,-0.640236,-0.361242
3,0,3,1.209981,15.374021,15.314907,2.326710,-0.071388,-0.047530,0.043787,-0.778621,-0.396796
4,0,4,1.211254,16.074363,16.017964,2.338976,-0.004316,-0.059042,0.066770,-0.841548,-0.404204
...,...,...,...,...,...,...,...,...,...,...,...
1874995,3124,595,1.009050,25.963234,25.897316,-0.830368,-0.745094,0.753563,-0.424747,-1.514932,-1.110041
1874996,3124,596,1.002827,25.784692,25.722482,-0.783942,-0.745265,0.809468,-0.436157,-1.482556,-1.114928
1874997,3124,597,1.006239,25.628060,25.572145,-0.755126,-0.758108,0.864391,-0.402916,-1.438439,-1.154946
1874998,3124,598,1.001038,25.626266,25.573288,-0.701296,-0.783199,0.879474,-0.341043,-1.443746,-1.168008


### Standard scaling 적용

In [44]:
from sklearn.preprocessing import StandardScaler,MinMaxScaler
col=train.columns
scaler = StandardScaler()

train.iloc[:,2:]= scaler.fit_transform(train.iloc[:,2:])
train = pd.DataFrame(data = train,columns =col)

test.iloc[:,2:]= scaler.transform(test.iloc[:,2:])
test = pd.DataFrame(data = test,columns =col)

In [45]:
train

,id,time,acc_Energy,gy_Energy,gy_acc_Energy,acc_x_norm,acc_y_norm,acc_z_norm,gy_x_norm,gy_y_norm,gy_z_norm
0,0,0,0.495681,-0.272719,-0.276391,2.307314,-0.029939,-0.432104,0.011776,-0.410859,-0.461007
1,0,1,0.742974,-0.236152,-0.240632,2.421086,-0.063321,-0.504058,0.026484,-0.528052,-0.365459
2,0,2,0.819822,-0.169815,-0.173080,2.444664,-0.056749,-0.654199,-0.037962,-0.596041,-0.366767
3,0,3,0.785669,-0.035229,-0.040560,2.428612,-0.116782,-0.573407,0.066099,-0.725311,-0.401232
4,0,4,0.791528,0.021954,0.016872,2.439475,-0.044220,-0.588886,0.091969,-0.784094,-0.408413
...,...,...,...,...,...,...,...,...,...,...,...
1874995,3124,595,-0.138940,0.829394,0.823900,-0.367432,-0.845648,0.503707,-0.461266,-1.413128,-1.092639
1874996,3124,596,-0.167578,0.814816,0.809618,-0.326315,-0.845833,0.578875,-0.474109,-1.382884,-1.097377
1874997,3124,597,-0.151875,0.802027,0.797338,-0.300794,-0.859728,0.652722,-0.436694,-1.341672,-1.136169
1874998,3124,598,-0.175811,0.801880,0.797431,-0.253120,-0.886873,0.673002,-0.367051,-1.346630,-1.148831


In [46]:
train.shape

(1875000, 11)

## 형 변환 하기

In [47]:
X=np.array(train.iloc[:,2:]).reshape(3125, 600, -1)

In [48]:
X.shape

(3125, 600, 9)

In [49]:
test_x=np.array(test.iloc[:,2:]).reshape(782, 600, -1)
test_x.shape

y = train_labels['label'].values
y = tf.keras.utils.to_categorical(train_labels['label']) 
y.shape

(3125, 61)

##### 모델링

+ CNN, LSTM, CNN+LSTM 등 여러 구조 적용해보다가 CNN에서 Flatten 없이 Global average pooling 한 구조가 가장 성능이 좋아 채택했습니다.


##### 모델 구조 

In [50]:
from sklearn.model_selection import KFold

In [52]:
prediction=0
folds=5
repeats=5
epochsize=40
drop=0.5
kfold = KFold(n_splits=folds, shuffle=True)

for k in range(repeats):
    for train_ind, test_ind in kfold.split(X, y):
        seed(2021)
        tf.random.set_seed(2021)

        input_layer = keras.layers.Input(X.shape[1:])
        conv1 = keras.layers.Conv1D(filters=128, kernel_size=9, padding='same')(input_layer)
        conv1 = keras.layers.BatchNormalization()(conv1)
        conv1 = keras.layers.Activation(activation='relu')(conv1)
        conv1 = keras.layers.Dropout(rate=0.3)(conv1)

        conv2 = keras.layers.Conv1D(filters=256, kernel_size=6, padding='same')(conv1)
        conv2 = keras.layers.BatchNormalization()(conv2)
        conv2 = keras.layers.Activation('relu')(conv2)
        conv2 = keras.layers.Dropout(rate=0.4)(conv2)

        conv3 = keras.layers.Conv1D(128, kernel_size=3,padding='same')(conv2)
        conv3 = keras.layers.BatchNormalization()(conv3)
        conv3 = keras.layers.Activation('relu')(conv3)
        conv3 = keras.layers.Dropout(rate=0.5)(conv3)

        gap = keras.layers.GlobalAveragePooling1D()(conv3)

        output_layer = keras.layers.Dense(61, activation='softmax')(gap)

        model = keras.models.Model(inputs=input_layer, outputs=output_layer)

        #adam = optimizers.Adam(lr=0.0001)

        model.compile(loss='categorical_crossentropy',
                      optimizer='Adam',
                      metrics=['accuracy'])

        model.fit(np.array(X)[train_ind], y[train_ind], epochs=epochsize, batch_size=64,
                  validation_data=(np.array(X)[test_ind], y[test_ind]))

        prediction += (model.predict(test_x)/(folds*repeats))
    print(f'\n\n********    {k+1}번 러닝 완료    *********\n\n')

submission.iloc[:,1:]=prediction
submission.to_csv('final_answer.csv', index=False)

Epoch 1/40
40/40 [==============================] - 5s 93ms/step - loss: 2.7597 - accuracy: 0.4764 - val_loss: 3.1768 - val_accuracy: 0.3120
Epoch 2/40
40/40 [==============================] - 3s 79ms/step - loss: 1.8293 - accuracy: 0.5860 - val_loss: 1.8816 - val_accuracy: 0.5504
Epoch 3/40
40/40 [==============================] - 3s 81ms/step - loss: 1.5780 - accuracy: 0.6104 - val_loss: 1.7055 - val_accuracy: 0.5600
Epoch 4/40
40/40 [==============================] - 3s 82ms/step - loss: 1.4177 - accuracy: 0.6300 - val_loss: 1.5142 - val_accuracy: 0.6176
Epoch 5/40
40/40 [==============================] - 3s 78ms/step - loss: 1.3151 - accuracy: 0.6528 - val_loss: 1.4509 - val_accuracy: 0.6144
Epoch 6/40
40/40 [==============================] - 3s 79ms/step - loss: 1.2006 - accuracy: 0.6812 - val_loss: 1.2895 - val_accuracy: 0.6352
Epoch 7/40
40/40 [==============================] - 3s 80ms/step - loss: 1.1112 - accuracy: 0.6980 - val_loss: 1.2074 - val_accuracy: 0.6624
Epoch 8/40
40

Epoch 19/40
40/40 [==============================] - 3s 79ms/step - loss: 0.6610 - accuracy: 0.8020 - val_loss: 0.7987 - val_accuracy: 0.7664
Epoch 20/40
40/40 [==============================] - 3s 80ms/step - loss: 0.6560 - accuracy: 0.8068 - val_loss: 0.7708 - val_accuracy: 0.7808
Epoch 21/40
40/40 [==============================] - 3s 78ms/step - loss: 0.6371 - accuracy: 0.8096 - val_loss: 0.8007 - val_accuracy: 0.7728
Epoch 22/40
40/40 [==============================] - 3s 80ms/step - loss: 0.6038 - accuracy: 0.8224 - val_loss: 0.7926 - val_accuracy: 0.7680
Epoch 23/40
40/40 [==============================] - 3s 80ms/step - loss: 0.6041 - accuracy: 0.8224 - val_loss: 0.8366 - val_accuracy: 0.7616
Epoch 24/40
40/40 [==============================] - 3s 82ms/step - loss: 0.6049 - accuracy: 0.8184 - val_loss: 0.7870 - val_accuracy: 0.7680
Epoch 25/40
40/40 [==============================] - 3s 78ms/step - loss: 0.5789 - accuracy: 0.8284 - val_loss: 0.7707 - val_accuracy: 0.7648
Epoch 

Epoch 37/40
40/40 [==============================] - 3s 79ms/step - loss: 0.4728 - accuracy: 0.8584 - val_loss: 0.6742 - val_accuracy: 0.7984
Epoch 38/40
40/40 [==============================] - 3s 80ms/step - loss: 0.4630 - accuracy: 0.8552 - val_loss: 0.6896 - val_accuracy: 0.8016
Epoch 39/40
40/40 [==============================] - 3s 81ms/step - loss: 0.4429 - accuracy: 0.8620 - val_loss: 0.6941 - val_accuracy: 0.8048
Epoch 40/40
40/40 [==============================] - 3s 77ms/step - loss: 0.4241 - accuracy: 0.8728 - val_loss: 0.6944 - val_accuracy: 0.7968
Epoch 1/40
40/40 [==============================] - 4s 86ms/step - loss: 2.7543 - accuracy: 0.4664 - val_loss: 2.9320 - val_accuracy: 0.4176
Epoch 2/40
40/40 [==============================] - 3s 80ms/step - loss: 1.8198 - accuracy: 0.5852 - val_loss: 1.8787 - val_accuracy: 0.5552
Epoch 3/40
40/40 [==============================] - 3s 82ms/step - loss: 1.5745 - accuracy: 0.6152 - val_loss: 1.7213 - val_accuracy: 0.5728
Epoch 4/4

Epoch 15/40
40/40 [==============================] - 3s 81ms/step - loss: 0.7565 - accuracy: 0.7796 - val_loss: 0.8125 - val_accuracy: 0.7696
Epoch 16/40
40/40 [==============================] - 3s 77ms/step - loss: 0.7243 - accuracy: 0.7904 - val_loss: 0.7929 - val_accuracy: 0.7760
Epoch 17/40
40/40 [==============================] - 3s 77ms/step - loss: 0.7267 - accuracy: 0.7920 - val_loss: 0.7663 - val_accuracy: 0.7824
Epoch 18/40
40/40 [==============================] - 3s 81ms/step - loss: 0.6659 - accuracy: 0.8044 - val_loss: 0.7619 - val_accuracy: 0.7776
Epoch 19/40
40/40 [==============================] - 3s 78ms/step - loss: 0.6535 - accuracy: 0.8072 - val_loss: 0.7531 - val_accuracy: 0.7888
Epoch 20/40
40/40 [==============================] - 3s 79ms/step - loss: 0.6359 - accuracy: 0.8172 - val_loss: 0.7452 - val_accuracy: 0.7920
Epoch 21/40
40/40 [==============================] - 3s 80ms/step - loss: 0.6324 - accuracy: 0.8140 - val_loss: 0.7773 - val_accuracy: 0.7776
Epoch 

40/40 [==============================] - 3s 80ms/step - loss: 0.5097 - accuracy: 0.8456 - val_loss: 0.6656 - val_accuracy: 0.8064
Epoch 33/40
40/40 [==============================] - 3s 78ms/step - loss: 0.5054 - accuracy: 0.8404 - val_loss: 0.6702 - val_accuracy: 0.8064
Epoch 34/40
40/40 [==============================] - 3s 79ms/step - loss: 0.4752 - accuracy: 0.8520 - val_loss: 0.6370 - val_accuracy: 0.8160
Epoch 35/40
40/40 [==============================] - 3s 80ms/step - loss: 0.4477 - accuracy: 0.8640 - val_loss: 0.6567 - val_accuracy: 0.8096
Epoch 36/40
40/40 [==============================] - 3s 77ms/step - loss: 0.4853 - accuracy: 0.8516 - val_loss: 0.6769 - val_accuracy: 0.8000
Epoch 37/40
40/40 [==============================] - 3s 77ms/step - loss: 0.4532 - accuracy: 0.8596 - val_loss: 0.6693 - val_accuracy: 0.8208
Epoch 38/40
40/40 [==============================] - 3s 80ms/step - loss: 0.4677 - accuracy: 0.8520 - val_loss: 0.6666 - val_accuracy: 0.8064
Epoch 39/40
40/40 

...

Epoch 32/40
40/40 [==============================] - 3s 78ms/step - loss: 0.4633 - accuracy: 0.8640 - val_loss: 0.7965 - val_accuracy: 0.7696
Epoch 33/40
40/40 [==============================] - 3s 79ms/step - loss: 0.5011 - accuracy: 0.8488 - val_loss: 0.7854 - val_accuracy: 0.7728
Epoch 34/40
40/40 [==============================] - 3s 81ms/step - loss: 0.5026 - accuracy: 0.8428 - val_loss: 0.8045 - val_accuracy: 0.7888
Epoch 35/40
40/40 [==============================] - 3s 82ms/step - loss: 0.4906 - accuracy: 0.8460 - val_loss: 0.7393 - val_accuracy: 0.7968
Epoch 36/40
40/40 [==============================] - 3s 78ms/step - loss: 0.4773 - accuracy: 0.8528 - val_loss: 0.7572 - val_accuracy: 0.7952
Epoch 37/40
40/40 [==============================] - 3s 79ms/step - loss: 0.4597 - accuracy: 0.8604 - val_loss: 0.7283 - val_accuracy: 0.7776
Epoch 38/40
40/40 [==============================] - 3s 80ms/step - loss: 0.4751 - accuracy: 0.8584 - val_loss: 0.7638 - val_accuracy: 0.7936
Epoch 

40/40 [==============================] - 3s 78ms/step - loss: 0.9699 - accuracy: 0.7300 - val_loss: 1.0465 - val_accuracy: 0.6944
Epoch 10/40
40/40 [==============================] - 3s 80ms/step - loss: 0.9310 - accuracy: 0.7408 - val_loss: 1.0151 - val_accuracy: 0.6960
Epoch 11/40
40/40 [==============================] - 3s 81ms/step - loss: 0.8811 - accuracy: 0.7500 - val_loss: 0.9386 - val_accuracy: 0.7472
Epoch 12/40
40/40 [==============================] - 3s 82ms/step - loss: 0.8376 - accuracy: 0.7684 - val_loss: 0.9448 - val_accuracy: 0.7376
Epoch 13/40
40/40 [==============================] - 3s 77ms/step - loss: 0.8168 - accuracy: 0.7680 - val_loss: 0.8922 - val_accuracy: 0.7360
Epoch 14/40
40/40 [==============================] - 3s 79ms/step - loss: 0.7941 - accuracy: 0.7636 - val_loss: 0.8804 - val_accuracy: 0.7392
Epoch 15/40
40/40 [==============================] - 3s 81ms/step - loss: 0.7375 - accuracy: 0.7900 - val_loss: 0.8415 - val_accuracy: 0.7552
Epoch 16/40
40/40 

Epoch 27/40
40/40 [==============================] - 3s 80ms/step - loss: 0.5936 - accuracy: 0.8216 - val_loss: 0.7234 - val_accuracy: 0.7840
Epoch 28/40
40/40 [==============================] - 3s 79ms/step - loss: 0.5468 - accuracy: 0.8344 - val_loss: 0.7324 - val_accuracy: 0.7808
Epoch 29/40
40/40 [==============================] - 3s 81ms/step - loss: 0.5237 - accuracy: 0.8400 - val_loss: 0.7263 - val_accuracy: 0.7920
Epoch 30/40
40/40 [==============================] - 3s 78ms/step - loss: 0.5286 - accuracy: 0.8384 - val_loss: 0.7349 - val_accuracy: 0.7792
Epoch 31/40
40/40 [==============================] - 3s 79ms/step - loss: 0.5054 - accuracy: 0.8440 - val_loss: 0.7282 - val_accuracy: 0.7824
Epoch 32/40
40/40 [==============================] - 3s 80ms/step - loss: 0.4755 - accuracy: 0.8576 - val_loss: 0.7050 - val_accuracy: 0.7888
Epoch 33/40
40/40 [==============================] - 3s 77ms/step - loss: 0.4830 - accuracy: 0.8516 - val_loss: 0.7126 - val_accuracy: 0.7952
Epoch 

Epoch 5/40
40/40 [==============================] - 3s 78ms/step - loss: 1.2879 - accuracy: 0.6640 - val_loss: 1.4441 - val_accuracy: 0.5952
Epoch 6/40
40/40 [==============================] - 3s 79ms/step - loss: 1.2032 - accuracy: 0.6864 - val_loss: 1.3002 - val_accuracy: 0.6448
Epoch 7/40
40/40 [==============================] - 3s 81ms/step - loss: 1.1056 - accuracy: 0.7012 - val_loss: 1.2673 - val_accuracy: 0.6240
Epoch 8/40
40/40 [==============================] - 3s 78ms/step - loss: 1.0423 - accuracy: 0.7160 - val_loss: 1.1788 - val_accuracy: 0.6688
Epoch 9/40
40/40 [==============================] - 3s 80ms/step - loss: 0.9688 - accuracy: 0.7364 - val_loss: 1.1065 - val_accuracy: 0.6832
Epoch 10/40
40/40 [==============================] - 3s 81ms/step - loss: 0.9256 - accuracy: 0.7444 - val_loss: 1.0995 - val_accuracy: 0.6656
Epoch 11/40
40/40 [==============================] - 3s 81ms/step - loss: 0.9028 - accuracy: 0.7468 - val_loss: 1.0528 - val_accuracy: 0.6944
Epoch 12/40

Epoch 23/40
40/40 [==============================] - 3s 79ms/step - loss: 0.6471 - accuracy: 0.8092 - val_loss: 0.7364 - val_accuracy: 0.7824
Epoch 24/40
40/40 [==============================] - 3s 81ms/step - loss: 0.6422 - accuracy: 0.8064 - val_loss: 0.6996 - val_accuracy: 0.8080
Epoch 25/40
40/40 [==============================] - 3s 77ms/step - loss: 0.5964 - accuracy: 0.8208 - val_loss: 0.6895 - val_accuracy: 0.8048
Epoch 26/40
40/40 [==============================] - 3s 76ms/step - loss: 0.5785 - accuracy: 0.8236 - val_loss: 0.6889 - val_accuracy: 0.8224
Epoch 27/40
40/40 [==============================] - 3s 80ms/step - loss: 0.5541 - accuracy: 0.8316 - val_loss: 0.6880 - val_accuracy: 0.8080
Epoch 28/40
40/40 [==============================] - 3s 79ms/step - loss: 0.5419 - accuracy: 0.8332 - val_loss: 0.7096 - val_accuracy: 0.8096
Epoch 29/40
40/40 [==============================] - 3s 78ms/step - loss: 0.5226 - accuracy: 0.8444 - val_loss: 0.6934 - val_accuracy: 0.8048
Epoch 

In [53]:
prediction=0
folds=5
repeats=5
epochsize=40
drop=0.5
kfold = KFold(n_splits=folds, shuffle=True)

for k in range(repeats):
    for train_ind, test_ind in kfold.split(X, y):
        model = Sequential()
        model.add(Conv1D (kernel_size=60, filters=256, strides=3, padding='valid',
                        kernel_initializer='he_uniform', input_shape=X.shape[1:],
                        activation='relu'))
        model.add(MaxPooling1D(1, padding = 'valid'))

        model.add(Conv1D(kernel_size=60, filters=128, activation='relu'))
        model.add(MaxPooling1D(1, padding = 'valid'))

        model.add(Conv1D(kernel_size=60, filters=64, activation='relu'))
        model.add(MaxPooling1D(1, padding = 'valid'))

        model.add(BatchNormalization())

        model.add(Dropout(drop))
        model.add(Flatten())
        model.add(Dense(1024, activation='relu'))
        model.add(BatchNormalization())

        model.add(Dropout(drop))
        model.add(Dense(1024, activation='relu'))
        model.add(BatchNormalization())

        model.add(Dropout(drop))
        model.add(Dense(1024, activation='relu'))
        model.add(BatchNormalization())


        model.add(Dropout(drop))
        model.add(Dense(61))
        model.add(Activation('softmax'))

        adam = optimizers.Adam(lr=0.0001)

        model.compile(loss='categorical_crossentropy',
                      optimizer=adam,
                      metrics=['accuracy'])

        model.fit(np.array(X)[train_ind], y[train_ind], epochs=epochsize, batch_size=64,
                  validation_data=(np.array(X)[test_ind], y[test_ind]))

        prediction += (model.predict(test_x)/(folds*repeats))
    
    print(f'********    {k}번 러닝 완료    *********')
submission.iloc[:,1:]=prediction
submission.to_csv('py_answer2.csv', index=False)

Epoch 1/40
40/40 [==============================] - 4s 53ms/step - loss: 4.9760 - accuracy: 0.1052 - val_loss: 9.4475 - val_accuracy: 0.0224
Epoch 2/40
40/40 [==============================] - 2s 39ms/step - loss: 3.7855 - accuracy: 0.3384 - val_loss: 7.0425 - val_accuracy: 0.0976
Epoch 3/40
40/40 [==============================] - 2s 43ms/step - loss: 3.4392 - accuracy: 0.3948 - val_loss: 6.7808 - val_accuracy: 0.1504
Epoch 4/40
40/40 [==============================] - 2s 40ms/step - loss: 3.3326 - accuracy: 0.4148 - val_loss: 3.9643 - val_accuracy: 0.3584
Epoch 5/40
40/40 [==============================] - 2s 42ms/step - loss: 3.1902 - accuracy: 0.4288 - val_loss: 2.5931 - val_accuracy: 0.4656
Epoch 6/40
40/40 [==============================] - 2s 40ms/step - loss: 3.0719 - accuracy: 0.4536 - val_loss: 2.7869 - val_accuracy: 0.4672
Epoch 7/40
40/40 [==============================] - 2s 44ms/step - loss: 3.0169 - accuracy: 0.4452 - val_loss: 2.4523 - val_accuracy: 0.4976
Epoch 8/40
40

Epoch 19/40
40/40 [==============================] - 2s 41ms/step - loss: 2.5531 - accuracy: 0.4864 - val_loss: 2.1160 - val_accuracy: 0.5408
Epoch 20/40
40/40 [==============================] - 2s 43ms/step - loss: 2.5963 - accuracy: 0.4840 - val_loss: 1.8796 - val_accuracy: 0.5600
Epoch 21/40
40/40 [==============================] - 2s 40ms/step - loss: 2.4542 - accuracy: 0.4888 - val_loss: 1.8714 - val_accuracy: 0.5712
Epoch 22/40
40/40 [==============================] - 2s 39ms/step - loss: 2.4423 - accuracy: 0.5008 - val_loss: 1.8554 - val_accuracy: 0.5568
Epoch 23/40
40/40 [==============================] - 2s 42ms/step - loss: 2.3851 - accuracy: 0.5032 - val_loss: 1.8983 - val_accuracy: 0.5520
Epoch 24/40
40/40 [==============================] - 2s 40ms/step - loss: 2.4526 - accuracy: 0.4916 - val_loss: 1.8213 - val_accuracy: 0.5536
Epoch 25/40
40/40 [==============================] - 2s 42ms/step - loss: 2.3868 - accuracy: 0.4980 - val_loss: 1.7839 - val_accuracy: 0.5696
Epoch 

Epoch 37/40
40/40 [==============================] - 2s 39ms/step - loss: 2.1372 - accuracy: 0.5276 - val_loss: 1.8204 - val_accuracy: 0.5680
Epoch 38/40
40/40 [==============================] - 2s 41ms/step - loss: 2.0551 - accuracy: 0.5324 - val_loss: 1.6511 - val_accuracy: 0.6032
Epoch 39/40
40/40 [==============================] - 2s 43ms/step - loss: 2.0864 - accuracy: 0.5364 - val_loss: 1.7781 - val_accuracy: 0.5984
Epoch 40/40
40/40 [==============================] - 2s 40ms/step - loss: 2.0140 - accuracy: 0.5404 - val_loss: 1.6600 - val_accuracy: 0.5952
Epoch 1/40
40/40 [==============================] - 3s 47ms/step - loss: 4.9147 - accuracy: 0.1140 - val_loss: 9.7969 - val_accuracy: 0.0128
Epoch 2/40
40/40 [==============================] - 2s 42ms/step - loss: 3.8023 - accuracy: 0.3400 - val_loss: 9.0677 - val_accuracy: 0.1248
Epoch 3/40
40/40 [==============================] - 2s 39ms/step - loss: 3.4543 - accuracy: 0.4116 - val_loss: 4.7824 - val_accuracy: 0.3264
Epoch 4/4

Epoch 15/40
40/40 [==============================] - 2s 45ms/step - loss: 2.5659 - accuracy: 0.4792 - val_loss: 1.8200 - val_accuracy: 0.5872
Epoch 16/40
40/40 [==============================] - 2s 40ms/step - loss: 2.5292 - accuracy: 0.4872 - val_loss: 1.8660 - val_accuracy: 0.5632
Epoch 17/40
40/40 [==============================] - 2s 41ms/step - loss: 2.7878 - accuracy: 0.4728 - val_loss: 2.0051 - val_accuracy: 0.5552
Epoch 18/40
40/40 [==============================] - 2s 45ms/step - loss: 2.5649 - accuracy: 0.4768 - val_loss: 2.1992 - val_accuracy: 0.5008
Epoch 19/40
40/40 [==============================] - 2s 41ms/step - loss: 2.4998 - accuracy: 0.4772 - val_loss: 2.0361 - val_accuracy: 0.5168
Epoch 20/40
40/40 [==============================] - 2s 44ms/step - loss: 2.5782 - accuracy: 0.4796 - val_loss: 2.1891 - val_accuracy: 0.5248
Epoch 21/40
40/40 [==============================] - 2s 39ms/step - loss: 2.3856 - accuracy: 0.4920 - val_loss: 2.0044 - val_accuracy: 0.5632
Epoch 

40/40 [==============================] - 2s 39ms/step - loss: 2.0771 - accuracy: 0.5352 - val_loss: 1.6145 - val_accuracy: 0.5840
Epoch 33/40
40/40 [==============================] - 2s 40ms/step - loss: 2.1653 - accuracy: 0.5280 - val_loss: 1.6706 - val_accuracy: 0.5680
Epoch 34/40
40/40 [==============================] - 2s 42ms/step - loss: 2.0666 - accuracy: 0.5312 - val_loss: 1.6814 - val_accuracy: 0.5888
Epoch 35/40
40/40 [==============================] - 2s 39ms/step - loss: 2.0448 - accuracy: 0.5488 - val_loss: 1.7233 - val_accuracy: 0.5696
Epoch 36/40
40/40 [==============================] - 2s 43ms/step - loss: 2.0089 - accuracy: 0.5412 - val_loss: 1.7998 - val_accuracy: 0.5600
Epoch 37/40
40/40 [==============================] - 2s 40ms/step - loss: 1.9402 - accuracy: 0.5476 - val_loss: 1.6761 - val_accuracy: 0.5776
Epoch 38/40
40/40 [==============================] - 2s 38ms/step - loss: 1.9506 - accuracy: 0.5520 - val_loss: 1.6723 - val_accuracy: 0.5776
Epoch 39/40
40/40 

...

Epoch 32/40
40/40 [==============================] - 2s 42ms/step - loss: 2.1560 - accuracy: 0.5240 - val_loss: 1.6384 - val_accuracy: 0.5984
Epoch 33/40
40/40 [==============================] - 2s 41ms/step - loss: 2.2503 - accuracy: 0.5076 - val_loss: 2.6314 - val_accuracy: 0.4368
Epoch 34/40
40/40 [==============================] - 2s 43ms/step - loss: 2.1308 - accuracy: 0.5228 - val_loss: 1.8295 - val_accuracy: 0.5696
Epoch 35/40
40/40 [==============================] - 2s 40ms/step - loss: 2.2830 - accuracy: 0.5040 - val_loss: 1.9284 - val_accuracy: 0.5472
Epoch 36/40
40/40 [==============================] - 2s 41ms/step - loss: 2.1491 - accuracy: 0.5280 - val_loss: 1.7661 - val_accuracy: 0.5600
Epoch 37/40
40/40 [==============================] - 2s 42ms/step - loss: 2.0731 - accuracy: 0.5340 - val_loss: 1.7079 - val_accuracy: 0.5680
Epoch 38/40
40/40 [==============================] - 2s 39ms/step - loss: 2.0200 - accuracy: 0.5500 - val_loss: 1.6820 - val_accuracy: 0.6000
Epoch 

40/40 [==============================] - 2s 39ms/step - loss: 2.8073 - accuracy: 0.4552 - val_loss: 2.0987 - val_accuracy: 0.5152
Epoch 10/40
40/40 [==============================] - 2s 43ms/step - loss: 2.7755 - accuracy: 0.4512 - val_loss: 2.2084 - val_accuracy: 0.4928
Epoch 11/40
40/40 [==============================] - 2s 40ms/step - loss: 2.6994 - accuracy: 0.4788 - val_loss: 2.0114 - val_accuracy: 0.5232
Epoch 12/40
40/40 [==============================] - 2s 44ms/step - loss: 2.6900 - accuracy: 0.4744 - val_loss: 1.8761 - val_accuracy: 0.5504
Epoch 13/40
40/40 [==============================] - 2s 39ms/step - loss: 2.5085 - accuracy: 0.4936 - val_loss: 1.7623 - val_accuracy: 0.5536
Epoch 14/40
40/40 [==============================] - 2s 40ms/step - loss: 2.5077 - accuracy: 0.4956 - val_loss: 1.8371 - val_accuracy: 0.5440
Epoch 15/40
40/40 [==============================] - 2s 42ms/step - loss: 2.4608 - accuracy: 0.4960 - val_loss: 1.8750 - val_accuracy: 0.5344
Epoch 16/40
40/40 

Epoch 27/40
40/40 [==============================] - 2s 39ms/step - loss: 2.2200 - accuracy: 0.5200 - val_loss: 1.7515 - val_accuracy: 0.5856
Epoch 28/40
40/40 [==============================] - 2s 43ms/step - loss: 2.2686 - accuracy: 0.5172 - val_loss: 1.9062 - val_accuracy: 0.5520
Epoch 29/40
40/40 [==============================] - 2s 40ms/step - loss: 2.1228 - accuracy: 0.5256 - val_loss: 1.8310 - val_accuracy: 0.5952
Epoch 30/40
40/40 [==============================] - 2s 44ms/step - loss: 2.1549 - accuracy: 0.5312 - val_loss: 1.6722 - val_accuracy: 0.5792
Epoch 31/40
40/40 [==============================] - 2s 39ms/step - loss: 2.1167 - accuracy: 0.5268 - val_loss: 1.6135 - val_accuracy: 0.5856
Epoch 32/40
40/40 [==============================] - 2s 40ms/step - loss: 2.1615 - accuracy: 0.5280 - val_loss: 1.7214 - val_accuracy: 0.5888
Epoch 33/40
40/40 [==============================] - 2s 42ms/step - loss: 2.1003 - accuracy: 0.5172 - val_loss: 1.7058 - val_accuracy: 0.5728
Epoch 

Epoch 5/40
40/40 [==============================] - 2s 40ms/step - loss: 3.2995 - accuracy: 0.4032 - val_loss: 3.9002 - val_accuracy: 0.3776
Epoch 6/40
40/40 [==============================] - 2s 44ms/step - loss: 3.1559 - accuracy: 0.4188 - val_loss: 3.4559 - val_accuracy: 0.4208
Epoch 7/40
40/40 [==============================] - 2s 42ms/step - loss: 3.0117 - accuracy: 0.4340 - val_loss: 2.8082 - val_accuracy: 0.4608
Epoch 8/40
40/40 [==============================] - 2s 42ms/step - loss: 2.8851 - accuracy: 0.4576 - val_loss: 2.5343 - val_accuracy: 0.4896
Epoch 9/40
40/40 [==============================] - 2s 40ms/step - loss: 2.9569 - accuracy: 0.4548 - val_loss: 2.2811 - val_accuracy: 0.5024
Epoch 10/40
40/40 [==============================] - 2s 44ms/step - loss: 2.8879 - accuracy: 0.4444 - val_loss: 2.1832 - val_accuracy: 0.5184
Epoch 11/40
40/40 [==============================] - 2s 41ms/step - loss: 2.7707 - accuracy: 0.4656 - val_loss: 2.2154 - val_accuracy: 0.5344
Epoch 12/40

Epoch 23/40
40/40 [==============================] - 2s 40ms/step - loss: 2.3858 - accuracy: 0.4976 - val_loss: 2.0583 - val_accuracy: 0.5552
Epoch 24/40
40/40 [==============================] - 2s 40ms/step - loss: 2.3985 - accuracy: 0.4976 - val_loss: 1.8635 - val_accuracy: 0.5776
Epoch 25/40
40/40 [==============================] - 2s 42ms/step - loss: 2.3279 - accuracy: 0.4996 - val_loss: 1.7071 - val_accuracy: 0.5840
Epoch 26/40
40/40 [==============================] - 2s 39ms/step - loss: 2.2640 - accuracy: 0.5080 - val_loss: 1.6965 - val_accuracy: 0.5904
Epoch 27/40
40/40 [==============================] - 2s 43ms/step - loss: 2.3033 - accuracy: 0.4960 - val_loss: 1.7373 - val_accuracy: 0.5792
Epoch 28/40
40/40 [==============================] - 2s 40ms/step - loss: 2.3124 - accuracy: 0.5116 - val_loss: 1.6511 - val_accuracy: 0.6000
Epoch 29/40
40/40 [==============================] - 2s 43ms/step - loss: 2.2004 - accuracy: 0.5128 - val_loss: 1.6704 - val_accuracy: 0.5904
Epoch 

In [ ]:
prediction=0
folds=5
repeats=5
epochsize=40
drop=0.5
kfold = KFold(n_splits=folds, shuffle=True)
es =EarlyStopping(monitor='val_loss', patience=10, mode='min')

for k in range(repeats):
    for train_ind, test_ind in kfold.split(X, y):
        model = Sequential()
        model.add(Conv1D (kernel_size=30, filters=256, strides=3, padding='valid',
                         kernel_initializer='he_uniform', input_shape=X.shape[1:],
                         activation='relu'))
        model.add(MaxPooling1D(1, padding = 'valid'))

        model.add(Conv1D(kernel_size=30, filters=128, activation='relu'))
        model.add(MaxPooling1D(1, padding = 'valid'))

        model.add(BatchNormalization())

        model.add(Dropout(drop))
        model.add(GlobalAveragePooling1D())
        model.add(Dense(512, activation='relu'))
        model.add(BatchNormalization())

        model.add(Dropout(drop))
        model.add(Dense(61))
        model.add(Activation('softmax'))

        adam = optimizers.Adam(lr=0.0001)

        model.compile(loss='categorical_crossentropy',
                      optimizer=adam,
                      metrics=['accuracy'])

        model.fit(np.array(X)[train_ind], y[train_ind], epochs=epochsize, batch_size=64,
                  validation_data=(np.array(X)[test_ind], y[test_ind]), callbacks = [es])

        prediction += (model.predict(test_x)/(folds*repeats))
    print(f'\n\n********    {k+1}번 러닝 완료    *********\n\n')
submission.iloc[:,1:]=prediction
submission.to_csv('py_answer3.csv', index=False)

##### 성능 확인 및 제출

In [ ]:
model1ans=pd.read_csv('./py_answer1.csv')
model2ans=pd.read_csv('./py_answer2.csv')
model3ans=pd.read_csv('./py_answer3.csv')

In [ ]:
final_sv=(model1ans+model2ans+model3ans)/3

In [ ]:
# final_sv.to_csv('/content/drive/MyDrive/월간데이콘11/py_new4.csv', index=False)

final_sv.to_csv('py_final_answer.csv', index=False)